## Finding Fake/Duplicate Records
Commonly known as Data Dedupication & Record Linkage
Ref: https://www.kaggle.com/code/caesarlupum/deduping-record-linkage

In [ ]:
# Download and unzip data
!wget http://data.insideairbnb.com/united-kingdom/england/london/2023-03-14/data/listings.csv.gz -P ../data/
!gunzip -kf ../data/listings.csv.gz

In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('stopwords')
nltktk.download('punkt')

In [1]:
from datetime import datetime, date
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import FeatureUnion, Pipeline
import re
import string
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer # or LancasterStemmer, RegexpStemmer, SnowballStemmer

default_stemmer = PorterStemmer()
default_stopwords = stopwords.words('english') # or any other list of your choice

%matplotlib inline

In [2]:
df = pd.read_csv('../data/listings.csv')

In [3]:
df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,714569379355913481,https://www.airbnb.com/rooms/714569379355913481,20230314070633,2023-03-14,city scrape,Lovely private bedroom in Muswell Hill.,Take a break and unwind at this peaceful oasis.,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,39009854,...,NaN,NaN,NaN,NaN,f,1,0,1,0,NaN
1,808038970516277767,https://www.airbnb.com/rooms/808038970516277767,20230314070633,2023-03-14,city scrape,Studio Flat Franklin London,Brand New Modern Kitchen<br />Close to Excelle...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,495977998,...,NaN,NaN,NaN,NaN,t,7,7,0,0,NaN
2,822557738577472503,https://www.airbnb.com/rooms/822557738577472503,20230314070633,2023-03-14,city scrape,PropertyPlug - 2Bed Flat in Edgware SmartTV WiFi,Enjoy easy access to everything from this perf...,NaN,https://a0.muscache.com/pictures/d77957d5-695a...,325629338,...,NaN,NaN,NaN,NaN,t,4,4,0,0,NaN
3,3518856,https://www.airbnb.com/rooms/3518856,20230314070633,2023-03-14,city scrape,Wimbledon Double Bedroom Ensuite,A welcoming and stylish 2 bedroom 2 bathroom f...,NaN,https://a0.muscache.com/pictures/23a18442-fc1d...,187811,...,5.0,3.67,3.67,NaN,f,2,0,2,0,0.05
4,4876550,https://www.airbnb.com/rooms/4876550,20230314070633,2023-03-14,city scrape,Stunning Apartment 2 minutes walk to Tube Station,Luxury Modern Apartment in modern development ...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,25087384,...,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN


In [4]:
# Since this is a data deduplication problem, we're interested in data uploaded by the host, let's print list of columns to find out those columns
df.shape

(75241, 75)

In [5]:
# Huge dataset - Take first 2000 rows for analysis
df = df.iloc[:2000]

In [6]:
df['host_location'].unique()

array(['London, United Kingdom', nan, 'England, United Kingdom',
       'South Croydon, United Kingdom', 'New Malden, United Kingdom',
       'Croydon, United Kingdom', 'Hatfield Heath, United Kingdom',
       'Bedford, United Kingdom', 'Edgware, United Kingdom',
       'United Kingdom', 'Purley, United Kingdom',
       'Carshalton, United Kingdom', 'Everett, MA',
       'Uxbridge, United Kingdom', 'Ayr, United Kingdom',
       'Ilford, United Kingdom', 'Panchgani, India',
       'Ruislip, United Kingdom', 'Cardiff, United Kingdom',
       'Perpignan, France', 'Courbevoie, France',
       'Feltham, United Kingdom', 'London, Canada',
       'Enfield, United Kingdom', 'Tokyo, Japan', 'San Ramon, CA',
       'Marlow, United Kingdom', 'Edinburgh, United Kingdom',
       'Leeds, United Kingdom', 'Baku, Azerbaijan', 'Brasilia, Brazil',
       'Hayes, United Kingdom', 'Dartford, United Kingdom',
       'Valencia, Spain', 'Weston, FL', 'Barcelona, Spain',
       'Rio de Janeiro, Brazil', 'Pari

In [7]:
selectedcols = ['name', 'description', 'neighborhood_overview', 'host_id',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365']

In [ ]:
df = df[selectedcols]
df.head()

In [ ]:
# Now lets first understand and work on numerical columns
df.describe()

In [ ]:
# drop nan columns
nan_cols = ['bathrooms',
           'neighbourhood_group_cleansed']
df.drop(nan_cols, axis=1, inplace=True)
df.describe()

In [ ]:
# Get list of text columns
objcols = df.select_dtypes('object').columns
objcols

In [ ]:
# Lets drop picture urls
drop_cols = ['host_thumbnail_url', 'host_picture_url']
df.drop(drop_cols, axis=1, inplace=True)

In [ ]:
# host_name
df['host_name'].str.lower().value_counts()

In [ ]:
no = df['neighborhood_overview'].astype('str')
len(no[0])

In [ ]:
# host_since
def age(born):
    born = datetime.strptime(born, "%Y-%m-%d").date()
    today = date.today()
    return today.year - born.year - ((today.month, 
                                      today.day) < (born.month, 
                                                    born.day))
df['age'] = df['host_since'].apply(age)
df['age']
df.drop('host_since', axis=1, inplace=True)
df.head()

In [ ]:
# host_location, Most of the values are similar so can be dropped
df['host_location'].value_counts()
df.drop('host_location', axis=1, inplace=True)

In [ ]:
# host_about

In [ ]:
# host_response_time
df['host_response_time'].value_counts()
df['host_response_time'].replace(['within an hour', 'within a few hours', 'within a day', 'a few days or more'],
                                [1, 2, 3, 4], inplace=True)
df['host_response_time'] = df['host_response_time'].astype('int64')

In [ ]:
# Lot of null values are there, but this can be important depictor of host behaviour and property, so let's impute
df['host_response_time'].isnull().value_counts()

In [ ]:
sns.boxplot(df.host_response_time)
#
# Distribution plot
#
sns.distplot(df.host_response_time)

In [ ]:
# Lets fill with median value
df['host_response_time'] = df['host_response_time'].fillna(df['host_response_time'].median())
df['host_response_time'].isnull().value_counts()

In [ ]:
# host_response_rate
df['host_response_rate'].isnull().value_counts()

In [ ]:
# host_acceptance_rate

In [ ]:
# host_is_superhost

## Let's create a simple dataset from property columns

In [8]:
selectedcols = ['name', 'description', 'neighborhood_overview', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365']

In [9]:
df = df[selectedcols]
df.head()

,name,description,neighborhood_overview,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,...,beds,amenities,price,minimum_nights,maximum_nights,has_availability,availability_30,availability_60,availability_90,availability_365
0,Lovely private bedroom in Muswell Hill.,Take a break and unwind at this peaceful oasis.,NaN,NaN,Haringey,NaN,51.597280,-0.139330,Private room in condo,Private room,...,1.0,"[""Iron"", ""Hangers"", ""Hair dryer"", ""Outdoor din...",$100.00,1,365,t,30,60,90,365
1,Studio Flat Franklin London,Brand New Modern Kitchen<br />Close to Excelle...,NaN,NaN,Barnet,NaN,51.636518,-0.177475,Entire rental unit,Entire home/apt,...,1.0,[],$65.00,180,365,t,30,60,90,365
2,PropertyPlug - 2Bed Flat in Edgware SmartTV WiFi,Enjoy easy access to everything from this perf...,NaN,NaN,Harrow,NaN,51.608180,-0.277400,Entire rental unit,Entire home/apt,...,2.0,"[""Dining table"", ""Washer"", ""Outdoor furniture""...",$132.00,2,28,t,12,20,35,35
3,Wimbledon Double Bedroom Ensuite,A welcoming and stylish 2 bedroom 2 bathroom f...,NaN,NaN,Merton,NaN,51.422310,-0.188410,Private room in rental unit,Private room,...,1.0,"[""Washer"", ""Iron"", ""Hangers"", ""Kitchen"", ""Smok...",$100.00,5,1125,t,30,60,90,365
4,Stunning Apartment 2 minutes walk to Tube Station,Luxury Modern Apartment in modern development ...,NaN,NaN,Barnet,NaN,51.602282,-0.193606,Entire condo,Entire home/apt,...,1.0,"[""First aid kit"", ""Washer"", ""Fire extinguisher...",$120.00,5,90,t,23,53,83,337


In [10]:
df.isna().sum()

name                               5
description                      126
neighborhood_overview           1392
neighbourhood                   1392
neighbourhood_cleansed             0
neighbourhood_group_cleansed    2000
latitude                           0
longitude                          0
property_type                      0
room_type                          0
accommodates                       0
bathrooms                       2000
bathrooms_text                    12
bedrooms                         108
beds                              36
amenities                          0
price                              0
minimum_nights                     0
maximum_nights                     0
has_availability                   0
availability_30                    0
availability_60                    0
availability_90                    0
availability_365                   0
dtype: int64

In [11]:
# drop nan columns
nan_cols = ['bathrooms',
           'neighbourhood_group_cleansed']
df.drop(nan_cols, axis=1, inplace=True)

In [12]:
# Almost 40-50% rows in neighborhood_overview, neighborhood so lets drop them
nan_cols = ['neighborhood_overview',
           'neighbourhood']
df.drop(nan_cols, axis=1, inplace=True)

In [13]:
df.isna().sum()

name                        5
description               126
neighbourhood_cleansed      0
latitude                    0
longitude                   0
property_type               0
room_type                   0
accommodates                0
bathrooms_text             12
bedrooms                  108
beds                       36
amenities                   0
price                       0
minimum_nights              0
maximum_nights              0
has_availability            0
availability_30             0
availability_60             0
availability_90             0
availability_365            0
dtype: int64

In [14]:
# Drop nan reocrds in name
df = df.dropna(subset=['name'])
df.isna().sum()

name                        0
description               125
neighbourhood_cleansed      0
latitude                    0
longitude                   0
property_type               0
room_type                   0
accommodates                0
bathrooms_text             12
bedrooms                  108
beds                       36
amenities                   0
price                       0
minimum_nights              0
maximum_nights              0
has_availability            0
availability_30             0
availability_60             0
availability_90             0
availability_365            0
dtype: int64

In [15]:
# Fillna
df['description'].fillna('', inplace=True)
df['bathrooms_text'].fillna('', inplace=True)
df['bedrooms'].fillna(df['bedrooms'].mean(), inplace=True)
df['beds'].fillna(df['beds'].mean(), inplace=True)

In [ ]:
def ngrams(text, n=3):
    text = text.encode("ascii", errors="ignore").decode() #remove non ascii chars
    text = text.lower()
    text = text.replace('&', ' and ')
    text = text.replace(',', ' ')
    text = text.replace('-', ' ')
#     chars_to_remove = [")","(",".","|","[","]","{","}","'"]
#     rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
#     text = re.sub(rx, '', text)
    text = [words for words in text if words not in string.punctuation]
    text = ' '.join(text)
    text = re.sub(' +',' ',text).strip() # get rid of multiple spaces and replace with a single
    text = ' '+ text +' ' # pad names for ngrams...
    text = re.sub(r'[,-./]|\sBD',r'', text)
    ngrams = zip(*[text[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]


In [ ]:
ngrams("Hello")

In [ ]:
property_names = df['name'].unique()
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(property_names)

In [ ]:
tf_idf_matrix.shape

In [ ]:
clean_text(df.iloc[0]['description'])

In [16]:
CLEANR = re.compile('<.*?>') 

def clean_series(inp_series):
    result = inp_series.apply(lambda x: clean_text(x))
    return result
    
def clean_text(text, ):

    def tokenize_text(text):
        return [w for s in sent_tokenize(text) for w in word_tokenize(s)]
    
    def remove_html_tags(text):
        return re.sub(CLEANR, '', text)

    def remove_special_characters(text, characters=string.punctuation.replace('-', '')):
        tokens = tokenize_text(text)
        pattern = re.compile('[{}]'.format(re.escape(characters)))
        return ' '.join(filter(None, [pattern.sub('', t) for t in tokens]))

    def stem_text(text, stemmer=default_stemmer):
        tokens = tokenize_text(text)
        return ' '.join([stemmer.stem(t) for t in tokens])

    def remove_stopwords(text, stop_words=default_stopwords):
        tokens = [w for w in tokenize_text(text) if w not in stop_words]
        return ' '.join(tokens)

    text = text.strip(' ') # strip whitespaces
    text = text.lower() # lowercase
    text = stem_text(text) # stemming
    text = remove_html_tags(text)
    text = remove_special_characters(text) # remove punctuation and symbols
    text = remove_stopwords(text) # remove stopwords
    #text.strip(' ') # strip whitespaces again?

    return text

In [17]:
transformer = FeatureUnion([
                ('name_tfidf', 
                  Pipeline([('extract_field',
                              FunctionTransformer(lambda x: x['name'], 
                                                  validate=False)),
                            ('cleaning', 
                              FunctionTransformer(clean_series)),
                            ('tfidf', 
                              TfidfVectorizer())])),
                ('description_tfidf', 
                  Pipeline([('extract_field', 
                              FunctionTransformer(lambda x: x['description'], 
                                                  validate=False)),
                            ('cleaning', 
                              FunctionTransformer(clean_series)),
                            ('tfidf', 
                              TfidfVectorizer())]))]) 

transformer.fit(df)



FeatureUnion(transformer_list=[('name_tfidf',
                                Pipeline(steps=[('extract_field',
                                                 FunctionTransformer(func=<function <lambda> at 0x7f56f6f79c10>)),
                                                ('cleaning',
                                                 FunctionTransformer(func=<function clean_series at 0x7f56f6f79e50>)),
                                                ('tfidf', TfidfVectorizer())])),
                               ('description_tfidf',
                                Pipeline(steps=[('extract_field',
                                                 FunctionTransformer(func=<function <lambda> at 0x7f56f6f79dc0>)),
                                                ('cleaning',
                                                 FunctionTransformer(func=<function clean_series at 0x7f56f6f79e50>)),
                                                ('tfidf',
                                        

In [18]:
search_vocab = transformer.transformer_list[0][1].steps[2][1].get_feature_names() 
product_vocab = transformer.transformer_list[1][1].steps[2][1].get_feature_names()
vocab = search_vocab + product_vocab

print(vocab)

['01', '010815', '03', '07776789835', '10', '105', '10min', '11', '114', '118', '119', '12', '1320', '14', '14a', '15', '150', '150mb', '15807', '15bath', '15min', '15minstooxfordstrt', '17', '1730', '173hpa', '179', '17th', '185', '18th', '19', '1920', '1bd', '1bdr', '1bed', '1bedflat', '1bedroom', '1br', '1min', '1person', '1ps', '1st', '20', '200', '2015', '20min', '21', '215', '21st', '22', '2206', '23', '247', '24hr', '24th', '25', '25min', '26ls', '27', '2bath', '2bd', '2bed', '2bedr3', '2bedroom', '2br', '2ep', '2f', '2nd', '2peopl', '2week', '30', '3006', '301', '30day', '31', '33', '35', '36', '37', '3a', '3bed', '3br', '3bth', '3c', '3e', '3eh', '3month', '3x3', '41', '42', '44', '45', '45bedroom', '46cm', '49', '4bdr', '4la', '4ppl', '4th', '50', '500m', '5bed', '5min', '5th', '60nt', '67', '6bd', '7di', '7min', '80', '910', '92', '9a', '9workspacesכשר', 'a13', 'abbey', 'abod', 'abov', 'abstract', 'ac', 'academi', 'access', 'accoglient', 'accommod', 'accomod', 'action', 'act

In [19]:
tfidf_matrix = transformer.transform(df).toarray()

In [20]:
print( tfidf_matrix.shape, tfidf_matrix[5] )

(1995, 7572) [0. 0. 0. ... 0. 0. 0.]


## Record Linkage using KNN

In [21]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(tfidf_matrix, test_size=0.2)

In [22]:
test.shape

(399, 7572)

In [23]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=1, n_jobs=-1, metric='cosine').fit(train)

In [24]:
distances, indices = nbrs.kneighbors(tfidf_matrix)

In [25]:
distances

array([[1.11022302e-16],
       [4.44089210e-16],
       [7.36062567e-01],
       ...,
       [0.00000000e+00],
       [6.37634556e-01],
       [3.33066907e-16]])

In [26]:
indices

array([[ 832],
       [ 681],
       [1539],
       ...,
       [ 362],
       [1414],
       [ 792]])

In [27]:
# For n_neighbors=1
matches = []
for i, j in enumerate(indices):
    temp = [round(distances[i][0], 2), df.iloc[i]['name'], df.iloc[j]['name'], j]
    matches.append(temp)

matches = pd.DataFrame(matches, columns=[
    'Match confidence (lower is better)',
    'Original Name',
    'Matched Name',
    'Matched Index'
])

In [28]:
matches

,Match confidence (lower is better),Original Name,Matched Name,Matched Index
0,0.00,Lovely private bedroom in Muswell Hill.,836 One-Bedroom Apartment in Nine Elms Zone...,[832]
1,0.00,Studio Flat Franklin London,"683 Super Appart, 20 s du metro Name: name,...",[681]
2,0.74,PropertyPlug - 2Bed Flat in Edgware SmartTV WiFi,"1543 Modern Stratford 2 bed pad Name: name,...",[1539]
3,0.00,Wimbledon Double Bedroom Ensuite,1413 Double Kind Room-Clerkenwell-July 24th...,[1409]
4,0.59,Stunning Apartment 2 minutes walk to Tube Station,454 Cozy studio in the heart of Dalston Nam...,[452]
...,...,...,...,...
1990,0.00,Loft flat in Notting Hill,1320 Luxury 2 Bedroom Apartment on the Edge...,[1316]
1991,0.00,School conversion in Central London,"342 Large Doble Room. Name: name, dtype: ob...",[342]
1992,0.00,En-suite Double Room in Twickenham,"364 Buscos villa Name: name, dtype: object",[362]
1993,0.64,beautiful well presented 3 bedrooms,1418 Chelsea Studio Flat Apartment - Cosy a...,[1414]


In [32]:
df.iloc[1994][['name', 'description', 'latitude', 'longitude']].values

array(['Double private room with bathroom',
       'Private bedroom with its own.m bathroom available in modern building, with gym, cinema and pool, close to subway/ bus (2min) and Canary Wharf (5 min walk) next to Hilton Hotel, with easy and fast connections to the City and Central London. Several supermarkets/dry cleaning/shops/restaurant around the corner.  Washing machine, TV and cleaning lady incl. Prefer professionals  in Canary Wharf/City for 10 weeks to 24 months internships/work secondments in London.',
       51.49743, -0.02018], dtype=object)

In [31]:
df.iloc[792][['name', 'description', 'latitude', 'longitude']].values

array(['Twin Room convenient for Heathrow. Own bathroom.',
       'Large twin room with warm terracotta decor and sunny aspect at the front of the house. This room would suit two friends or siblings wishing to share a room but have their own single bed and have exclusive use of the bathroom with shower and bath. We are very close to South Ealing station and so is handy for Hammersmith and Central London as well as Heathrow. We have two friendly cats who are not allowed in the bedrooms! Lovely cafes and parks nearby. Parking is available at a low daily cost.',
       51.50006, -0.30962], dtype=object)

In [ ]:
import math
a = df.iloc[6808]['latitude'] - df.iloc[4318]['latitude']
b = df.iloc[6808]['longitude'] - df.iloc[4318]['longitude']
math.sqrt((a*a) - (b*b))

In [ ]:
for i, j in enumerate(indices):
    if i < 15:
        continue
    temp = [round(distances[i][0], 2), df.iloc[i]['name'], df.iloc[j]['name'], j]
    print('Distance', round(distances[i][0], 2))
    print('Original Record')
    print(df.iloc[i]['name'])
    print(df.iloc[i]['description'])
    print('Matched Record')
    print(df.iloc[j]['name'])
    print(df.iloc[j]['description'])
    print('Matched Index', j)
    if i == 20:
        break

In [ ]:
df['latitude'].dtype

In [ ]:
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]
to_drop

In [ ]:
# Drop columns id, scrape_id, maximum_minimum_nights, maximum_maximum_nights, minimum_nights_avg_ntm, maximum_nights_avg_ntm, last_scraped
columns = ['id', 
           'scrape_id', 
           'maximum_minimum_nights', 
           'maximum_maximum_nights', 
           'minimum_nights_avg_ntm', 
           'maximum_nights_avg_ntm',
          'last_scraped']
df = df.drop(columns, axis=1)
df

In [ ]:
df.dtypes

In [ ]:
numerics = ['int64', 'float64']
numeric_columns = df.select_dtypes(include=numerics).columns

In [ ]:
numeric_columns

In [ ]:
# Analyze host_listings_count
df['host_listings_count'].value_counts()